In [5]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import time
import os

In [8]:
climbs = pd.read_csv("route-finder.csv")

In [11]:
climbs.head()

,Route,Location,URL,Avg Stars,Your Stars,Route Type,Rating,Pitches,Length,Area Latitude,Area Longitude
0,Unknown 5,West Slabs > Silver Plume > Colorado,https://www.mountainproject.com/route/10825985...,1.0,-1,"Trad, Sport",5.5,1,50.0,39.698,-105.7435
1,Unknown 7+,West Slabs > Silver Plume > Colorado,https://www.mountainproject.com/route/10825977...,1.5,-1,"Trad, Sport",5.7+,1,85.0,39.698,-105.7435
2,Unknown 9,West Slabs > Silver Plume > Colorado,https://www.mountainproject.com/route/10825981...,2.0,-1,"Trad, Sport",5.9,1,90.0,39.698,-105.7435
3,Unknown 2nd 9,West Slabs > Silver Plume > Colorado,https://www.mountainproject.com/route/10825981...,2.0,-1,"Trad, Sport",5.9,1,90.0,39.698,-105.7435
4,Unknown 9+,West Slabs > Silver Plume > Colorado,https://www.mountainproject.com/route/10825983...,2.0,-1,"Trad, Sport",5.9+,3,180.0,39.698,-105.7435


In [12]:
url = "https://www.mountainproject.com/gyms/utah"

In [14]:
r = requests.get(url)

In [19]:
soup = BeautifulSoup(r.text)

In [51]:
links = soup.find_all('a')
#links = [i['href'] for i in links]
#links[0]['href']

def hasGym(link):
    try:
        if "gym/" in link["href"]:
            return True
        else:
            return False
    except:
        return False

gyms = [link.text for link in links if hasGym(link)]
gyms

['Add Missing Gym',
 'AlpenFit - Indoor Training for Outdoo…',
 'Contact Climbing',
 'Dimple Dell',
 'Double-Diamond Climbing Wall',
 'Elevation Rock Gym',
 'Fit Stop Rock Climbing Gym',
 'iROCK Utah',
 'Lehi Legacy Center',
 'Momentum Indoor Climbing Gym',
 'Momentum Lehi',
 'Momentum Millcreek',
 'Salt Lake Bouldering Project',
 'SUU Climbing Gym',
 'Taylorsville Rec Center',
 'Technical Training Center',
 'The Front Climbing Club',
 'The Front Climbing Club',
 'The Front Climbing Club - South Main',
 'The Project',
 'The Quarry',
 'The Scratch Pad',
 'The Summit',
 'Uintah Recreation Community Center',
 'Ultimate Sports Arena',
 'Washington City Community Center',
 'West Valley Family Fitness Center cli…']